<a href="https://colab.research.google.com/github/drishtigulenia/data-science-Implementations/blob/main/linearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [200]:
# !pip install --upgrade quandl
import pandas as pd
import math
import numpy as np
from sklearn import preprocessing, model_selection, svm
from sklearn.linear_model import LinearRegression
# import quandl

In [201]:
# df = quandl.get('WIKI/GOOGL')
df = pd.read_csv('sample_data/banknifty.csv')
df.drop_duplicates(subset=['date'], keep='last', inplace=True)
# df['index'] = range(1, len(df) + 1)
df['HL_PCT'] = ((df['high'] - df['low']) / df['low']) * 100
df['PCT_change'] = ((df['close'] - df['open']) / df['open']) * 100
df.drop(['open', 'high', 'low','index', 'time', 'date'] , axis=1, inplace=True)
df.head()

,close,HL_PCT,PCT_change
374,12099.75,0.067381,-0.031396
749,12182.10,0.095267,-0.037336
1124,12289.20,0.047614,-0.024812
1499,12479.80,0.050103,0.029657
1874,12337.05,0.064440,-0.028767


In [203]:
forecast_out = -10;
forecast_in = 'close'
# df.fillna(-99999, inplace=True)
df['label'] = df[forecast_in].shift(forecast_out)
df.dropna(inplace=True)
df.head()
# len(df)

,close,HL_PCT,PCT_change,label
374,12099.75,0.067381,-0.031396,12418.10
749,12182.10,0.095267,-0.037336,12477.80
1124,12289.20,0.047614,-0.024812,12487.30
1499,12479.80,0.050103,0.029657,12458.00
1874,12337.05,0.064440,-0.028767,12307.65


In [204]:
X = np.array(df.drop(['label'], 1))
y = np.array(df['label'])
X = preprocessing.scale(X)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.2)
clf = LinearRegression()
clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)
print(accuracy)

0.33326323394787294
